In [1]:
#This code produces a plot to solve Homework 3 assigned by Carl Brune to the TALENT Nuclear Reaction Theory
#Summer School in 2019
#Author:  Chad Ummel

import math
import matplotlib.pyplot as plt
import numpy as np
import mpmath as mp

In [2]:
#Physical constants
hbar = 197.3286 #(hbar*c in MeV fm)
alpha = 1/137 #Fine structure constant

In [3]:
#Particle info

#Channel d = d+t ->Index 0
#Channel a = a+n ->Index 1

#Masses (in MeV)
mp = [2.013553212745*931.5,939.565] #projectile mass [m_d,m_n]
mt = [3.0160492*931.5,3727.79378] #target mass [m_t,m_a]
mu = np.multiply(mp,mt)/np.add(mp,mt)

#Charges
Zp = [1,0] #projectile charges [Z_d,Z_n]
Zt = [1,2] #target charges [Z_t,Z-a]

#Spins
Jp = [1,1/2] #Projectile Spins [J_d,J_n]
Jt = [1/2,0] #Target spins [J_t,J_a]

#Threshold Energies (in MeV)
Q=[0,17.589293]

In [4]:
#Constants defined by problem
ac = [5.0,3.0]
B = [-0.27864,-0.557]
l=[0,2]
gamma=[[0.95838,0.48304],[0.27781,1.51753]] #[[gamma_d_0,gamma_d_1],[gamma_a_0,gamma_a,1]
ER = [0.021626,10.0]
J = 3/2

In [5]:
#Coulomb subroutine
#NOT WORKING YET
def coulomb(l,k,ac,eta):
    rhoc = np.multiply(k,ac)
    space = round(np.log10(rhoc))-1
    meshpts = 1000
    rho = np.linspace(rhoc-space,rhoc+space,meshpts+1)
    F = mp.coulombf(l,eta,rho)
    G = mp.coulombg(l,eta,rho)
    Fc = F[int(meshpts/2)]
    Gc = G[int(meshpts/2)]
    A = np.sqrt(F*F + G*G)
    Ac = A[int(meshpts/2)]
    Aprime = np.gradient(A)
    Acprime = Aprime[int(meshpts/2)]
    return(rhoc,Ac,Acprime)

In [6]:
#Whittaker subroutine
#NOT WORKING YET
def whittaker(l,kappa,ac,etaneg):
    rhoc = 2*kappa*ac
    space = round(np.log10(rhoc))-1
    meshpts = 1000
    rho = np.linspace(rhoc-space,rhoc+space,meshpts+1)
    W=mp.whitw(etaneg,l+1/2,rho)
    Wc = W[int(meshpts/2)]
    Wprime = np.gradient(W)
    Wcprime = Wprime[int(meshpts/2)]
    return(rhoc,Wc,Wcprime)

In [7]:
#Penetration subroutine
def penetration(ECM,ac,l,k,eta):
    if ECM < 0:
        pen = 0.0
    else:
        (rhoc,Ac,Acprime)=coulomb(l,k,ac,eta)
        P = rhoc/Ac
        return(P)

In [8]:
#Shift subroutine
def shift(ECM,ac,l,k,kappa,eta,etaneg):
    if ECM < 0:
        (rhoc,Wc,Wcprime)=whittaker(l,kappa,ac,etaneg)
        Sc = rhoc/Wc*Wcprime
    else:
        (rhoc,Ac,Acprime)=coulomb(l,k,ac,eta)
        Sc = rhoc*Acprime/Ac
        return(Sc)

In [9]:
#R-matrix subroutine
def Rmatrix(E):
    
    E = E/1000 #Convert to MeV
    ECM = [0,0]
    ECM[0] = E*mt[0]/(mp[0]+mt[0])/1000
    ECM[1] = E*mt[1]/(mp[1]+mt[1])/1000
    kpre = np.sqrt(2*mu)/hbar
    k = kpre*np.sqrt(ECM)
    kappa = kpre*np.sqrt(np.subtract(0,ECM))
    etapre = alpha*np.multiply(Zp,Zt)*np.sqrt(mu/2);
    eta = etapre/np.sqrt(ECM)
    etaneg = -1*etapre/np.sqrt(np.subtract(0,ECM))

    #Calculate g-factors
    g = [0,0]
    g[0] = (2*J+1)/((2*Jp[0]+1)*(2*Jt[0]+1))
    g[1] = (2*J+1)/((2*Jp[1]+1)*(2*Jt[0]+1))

    #Calculate Excitation Energies
    Ex = ECM[0]-Q[0]

    #Calculate the R-matrix boi
    L=[0,0]
    R = [[0,0],[0,0]]
    R1 = [[0,0],[0,0]]
    P = [0,0]
    Sc = [0,0]
    P[0]=penetration(ECM[0],ac[0],l[0],k[0],eta[0])
    P[1]=penetration(ECM[1],ac[1],l[1],k[1],eta[1])
    Sc[0] =shift(ECM[0],ac[0],l[0],k[0],kappa[0],eta[0],etaneg[0])
    Sc[1] =shift(ECM[1],ac[1],l[1],k[1],kappa[1],eta[1],etaneg[1])
    for i in range(0,2,1):
        for j in range(0,2,1):
            for level in range(0,2,1):
                R[i][j] = R[i][j] + gamma[i][level]*gamma[j][level]/(ER[level]-Ex)
            L[j] = complex(0,0)#Sc[j],P[j])
            R1[i][j] = -1*R[i][j]*(L[j]-B[j])
            if i==j:
                R1[i][j] = R1[i][j] + 1
    
    RL = np.matmul(np.linalg.inv(R1),R)
    
    cs = math.pi/k[0]**2*g[0]*4*P[0]*P[1]*abs(RL[0][1]**2)/100
    S = cs*ECM[0]*math.exp(2*math.pi*eta[0])
    
    return(cs,S)

In [10]:
(cs,S)=Rmatrix(100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in sqrt
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in sqrt
  del sys.path[0]


AttributeError: 'list' object has no attribute 'coulombf'